In [1]:
# Standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sunpy.net import Fido, attrs
from sunpy.net.fido_factory import UnifiedResponse

# Custom modules
from jpm_logger import JpmLogger

__author__ = 'James Paul Mason'
__contact__ = 'jmason86@gmail.com'


def get_sdo_eve_lines_data(start_time, end_time, 
                           verbose=False):
    """Get emission lines data product from the Solar Dynamics Observatory / Extreme Ultraviolet Variability Experiment.

    Inputs:
        start_time [metatime]: The beginning of the time window of interest. See jpm_time_conversions.py
                               (https://github.com/jmason86/python_convenience_functions/blob/master/jpm_time_conversions.py)
                               for allowed metatime formats.
        end_time [metatime]:   Same as start time but for the end of the time window.
        
    Optional Inputs:
        verbose [bool]: Set to log the processing messages to disk and console. Default is False.

    Outputs:
        TBD
                                                 
    Optional Outputs:
        None

    Example:
        TBD
    """
    return 1

In [2]:
# prototype only: setup for test case
start_time = "2010-09-05 00:00:00"
end_time = "2010-09-05 01:00:00"

In [3]:
results = Fido.search(attrs.Time("2010-09-05 00:00:00", "2010-09-05 01:00:00"), attrs.Instrument('eve'))
results

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2010-09-05 00:00:00,2010-09-06 00:00:00,SDO,EVE,FULLDISK,1.0 .. 304.0
2010-09-05 00:00:00,2010-09-05 01:00:00,SDO,EVE,FULLDISK,93.0 .. 1033.0
2010-09-05 01:00:00,2010-09-05 02:00:00,SDO,EVE,FULLDISK,93.0 .. 1033.0
2010-09-05 00:00:00,2010-09-05 01:00:00,SDO,EVE,FULLDISK,60.0 .. 1060.0
2010-09-05 01:00:00,2010-09-05 02:00:00,SDO,EVE,FULLDISK,60.0 .. 1060.0
2010-09-05 00:00:00,2010-09-06 00:00:00,SDO,EVE,FULLDISK,1.0 .. 1050.0


The lines are the ones that have wavelength range [93.0 .. 1033.0] so I should be able to use wavemin to filter for them. But as you can see below, that doesn't actually work. It still returns the spectra (wavelength range [60.0 .. 1060.0], which makes no sense because that's not consistent with the input filter criteria. 
I've contacted Joe Gurman of the VSO at GSFC about this. 

In [4]:
from astropy import units as u
results_lines_only = Fido.search(attrs.Time(start_time, end_time), attrs.Instrument('eve'), attrs.Wavelength(wavemin=62 * u.angstrom, wavemax=1035 * u.angstrom))
results_lines_only

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2010-09-05 00:00:00,2010-09-05 01:00:00,SDO,EVE,FULLDISK,60.0 .. 1060.0
2010-09-05 01:00:00,2010-09-05 02:00:00,SDO,EVE,FULLDISK,60.0 .. 1060.0
2010-09-05 00:00:00,2010-09-06 00:00:00,SDO,EVE,FULLDISK,1.0 .. 1050.0


In [5]:
# Get the direct response from the VSO and filter for just the lines product
vso_responses = list(results.responses)
toss_indices = []
for i in range(0, len(vso_responses[0])):
    if 'Lines' not in vso_responses[0][i]['info']:
        toss_indices.append(i)
for toss_index in sorted(toss_indices, reverse=True):
    del vso_responses[0][toss_index]

In [6]:
# Convert the filtered response back to the UnifiedResponse and download
results = UnifiedResponse(vso_responses)
files = Fido.fetch(results, path='./{instrument}/{info}/{file}.fits')

[                                        ===============================

In [7]:
# Read the files and merge them into a single Dataframe

In [8]:
from sunpy import io

In [9]:
yar = io.read_file(files[0])

In [11]:
np.shape(yar)

(7, 2)

In [12]:
type(yar)

list

In [41]:
hdr = yar[5][1]
hdr.keys()

odict_keys(['XTENSION', 'BITPIX', 'NAXIS', 'NAXIS1', 'NAXIS2', 'PCOUNT', 'GCOUNT', 'TFIELDS', 'COMMENT', 'ORIGIN', 'DATE', 'TAI_OBS', 'DATE_OBS', 'T_OBS', 'EXPTIME', 'TIME', 'TELESCOP', 'INSTRUME', 'VERSION', 'REVISION', 'FILENAME', 'TTYPE1', 'TTYPE2', 'TTYPE3', 'TTYPE4', 'TTYPE5', 'TTYPE6', 'TTYPE7', 'TTYPE8', 'TTYPE9', 'TTYPE10', 'TTYPE11', 'TTYPE12', 'TTYPE13', 'TTYPE14', 'TTYPE15', 'TTYPE16', 'TTYPE17', 'TTYPE18', 'TTYPE19', 'TFORM1', 'TFORM2', 'TFORM3', 'TFORM4', 'TFORM5', 'TFORM6', 'TFORM7', 'TFORM8', 'TFORM9', 'TFORM10', 'TFORM11', 'TFORM12', 'TFORM13', 'TFORM14', 'TFORM15', 'TFORM16', 'TFORM17', 'TFORM18', 'TFORM19', 'EXTNAME', 'TUNIT6', 'TUNIT12', 'HISTORY', 'KEYCOMMENTS', 'WAVEUNIT'])

In [76]:
data = yar[5].data

In [77]:
type(data)

astropy.io.fits.fitsrec.FITS_rec

In [78]:
data.shape

(360,)

In [145]:
tai = data[2][0]
yyyydoy = data[2][1]
sod = data[2][2]
sc_flags_maybe = data[2][3]
flags_maybe = data[2][4]
line_irradiance = data[2][5]
line_precision = data[2][6]
line_accuracy = data[2][7]
band_irradiance = data[2][8]
band_precision = data[2][9]
band_accuracy = data[2][10]
diode_irradiance = data[2][11]
diode_stdev = data[2][12]
diode_precision = data[2][13]
diode_accuracy = data[2][14]
quad_fraction = data[2][15]
quad_stdev = data[2][16]
quad_precision = data[2][17]
quad_accuracy = data[2][18]

In [114]:
hm = data.data

In [116]:
print(hm[0])

NotImplementedError: memoryview: unsupported format T{>d:TAI:i:YYYYDOY:d:SOD:B:FLAGS:B:SC_FLAGS:(39)f:LINE_IRRADIANCE:(39)f:LINE_PRECISION:(39)f:LINE_ACCURACY:(20)f:BAND_IRRADIANCE:(20)f:BAND_PRECISION:(20)f:BAND_ACCURACY:(6)f:DIODE_IRRADIANCE:(6)f:DIODE_STDEV:(6)f:DIODE_PRECISION:(6)f:DIODE_ACCURACY:(4)f:QUAD_FRACTION:(4)f:QUAD_STDEV:(4)f:QUAD_PRECISION:(4)f:QUAD_ACCURACY:}

In [57]:
hdr = bla[5].header

In [33]:
hdr.keys()

odict_keys(['XTENSION', 'BITPIX', 'NAXIS', 'NAXIS1', 'NAXIS2', 'PCOUNT', 'GCOUNT', 'TFIELDS', 'COMMENT', 'ORIGIN', 'DATE', 'TAI_OBS', 'DATE_OBS', 'T_OBS', 'EXPTIME', 'TIME', 'TELESCOP', 'INSTRUME', 'VERSION', 'REVISION', 'FILENAME', 'TTYPE1', 'TTYPE2', 'TTYPE3', 'TTYPE4', 'TTYPE5', 'TTYPE6', 'TTYPE7', 'TTYPE8', 'TTYPE9', 'TTYPE10', 'TTYPE11', 'TTYPE12', 'TTYPE13', 'TTYPE14', 'TTYPE15', 'TTYPE16', 'TTYPE17', 'TTYPE18', 'TTYPE19', 'TFORM1', 'TFORM2', 'TFORM3', 'TFORM4', 'TFORM5', 'TFORM6', 'TFORM7', 'TFORM8', 'TFORM9', 'TFORM10', 'TFORM11', 'TFORM12', 'TFORM13', 'TFORM14', 'TFORM15', 'TFORM16', 'TFORM17', 'TFORM18', 'TFORM19', 'EXTNAME', 'TUNIT6', 'TUNIT12', 'HISTORY', 'KEYCOMMENTS', 'WAVEUNIT'])

In [139]:
data = bla[5].data

In [145]:
data[0][1]

2010248

In [73]:
len(data)

360

In [17]:
import sunpy.timeseries
import sunpy.data.sample
lyra = sunpy.timeseries.TimeSeries(sunpy.data.sample.LYRA_LEVEL3_TIMESERIES)

In [18]:
type(lyra)

sunpy.timeseries.sources.lyra.LYRATimeSeries

In [28]:
lyra.data.head()

,CHANNEL1,CHANNEL2,CHANNEL3,CHANNEL4
2011-06-07 00:00:00.010,0.006365,0.695866,0.002392,0.001100
2011-06-07 00:01:00.010,0.006365,0.695868,0.002392,0.001100
2011-06-07 00:02:00.010,0.006366,0.695870,0.002391,0.001100
2011-06-07 00:03:00.010,0.006366,0.695870,0.002391,0.001099
2011-06-07 00:04:00.010,0.006365,0.695873,0.002391,0.001099


In [29]:
rhessi = sunpy.timeseries.TimeSeries(sunpy.data.sample.RHESSI_TIMESERIES)

In [31]:
rhessi.data.head()

,3 - 6 keV,6 - 12 keV,12 - 25 keV,25 - 50 keV,50 - 100 keV,100 - 300 keV,300 - 800 keV,800 - 7000 keV,7000 - 20000 keV
2011-06-07 00:00:00,9,18,15,20,48,136,112,88,3
2011-06-07 00:00:04,8,18,16,22,48,136,108,88,4
2011-06-07 00:00:08,8,16,16,20,48,136,112,88,4
2011-06-07 00:00:12,8,20,18,22,48,136,112,88,4
2011-06-07 00:00:16,9,16,15,22,44,136,112,88,4


In [32]:
type(rhdaessi)

sunpy.timeseries.sources.rhessi.RHESSISummaryTimeSeries

In [ ]:
# Optionally average the data

In [ ]:
# Optionally change the missing data flag from -1 to user desired value

In [ ]:
# Return the EVE lines data with any optional modifications
return tbd